In [ ]:
import glob
import numpy as np
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
def get_data_from_fof_folder(output_dir, snapN, param1, param2):
    snapdir = glob.glob(output_dir+f"output/groups_*{snapN}")[0]
    snap_files = os.listdir(snapdir)
    
    all_values = None

    for file_name in snap_files:
        file_path = snapdir+f"/{file_name}"

        with h5py.File(file_path, "r") as f:
            values_this_file = f[f'{param1}/{param2}'][:]
            
            if all_values is None:
                all_values = values_this_file
            else:
                all_values = np.concatenate((all_values, values_this_file))
            
    
    return all_values

def get_data_from_snap_folder(output_dir, snapN, param1, param2):
    snapdir = glob.glob(output_dir+f"output/snapdir_*{snapN}")[0]
    snap_files = os.listdir(snapdir)
    # print(snap_files)
    
    all_values = None

    for file_name in snap_files:
        # print("-------------- File begin ----------------")
        file_path = snapdir+f"/{file_name}"
        # print(file_path)

        with h5py.File(file_path, "r") as f:
            # header = f['Header']
            # for key, val in header.attrs.items():
                # print(f"{key}: {val}")
            values_this_file = f[f'{param1}/{param2}'][:]
            
            if all_values is None:
                all_values = values_this_file
            else:
                all_values = np.concatenate((all_values, values_this_file))
            
            # print(values_this_file.shape)
            # print(all_values.shape)
        
        # print("--------------- File end ------------------")
    
    return all_values

def get_data_from_header(output_dir, snapN, param='BoxSize'):

    snapdir = glob.glob(output_dir+f"output/snapdir_*{snapN}")[0]
    snap_files = os.listdir(snapdir)
    file_name = snap_files[0]
    file_path = snapdir+f"/{file_name}"

    with h5py.File(file_path, "r") as f:
        header = f['Header']
        return_param = header.attrs[param]
        
    return return_param


def simple_hist_plot_old(path, snapN, particle_type="PartType4", zoom_out_factor=1, resolution=128, mask_zeros=False):
    
    mask_zeros_value = 0.7 # set zero bins to this value, so that the log colorbar works
    
    coordinates = get_data_from_snap_folder(path, snapN, particle_type, "Coordinates")
    masses = get_data_from_snap_folder(path, snapN, particle_type, "Masses")*1e10 # in M_sun
    
    box_size = get_data_from_header(path, snapN, 'BoxSize')/1e3 # in Mpc
    physical_bin_volume = (box_size/resolution)**2 * box_size  # this is a projection, so one axis has the full length of the box
    
    ############# Zoom in Filter ###############
    
    subhalo_mass = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloMass") * 1e10 # in M_sun
    M_milkyway = 1e12

    index_milkyway_like = min(range(len(subhalo_mass)), key=lambda i: abs(subhalo_mass[i] - M_milkyway))
    M_milkyway_like = subhalo_mass[index_milkyway_like]
    # print(index_milkyway_like, M_milkyway_like)

    subhalo_cms = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloCM")
    # print(subhalo_cms[index_milkyway_like])

    subhalo_hmr = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloHalfmassRad")
    # print(subhalo_hmr[index_milkyway_like])
    
    zoom_in_x = [subhalo_cms[index_milkyway_like][0] - (subhalo_hmr[index_milkyway_like]*zoom_out_factor), 
                 subhalo_cms[index_milkyway_like][0] + (subhalo_hmr[index_milkyway_like]*zoom_out_factor)]
    zoom_in_y = [subhalo_cms[index_milkyway_like][1] - (subhalo_hmr[index_milkyway_like]*zoom_out_factor), 
                 subhalo_cms[index_milkyway_like][1] + (subhalo_hmr[index_milkyway_like]*zoom_out_factor)]
    zoom_in_z = [subhalo_cms[index_milkyway_like][2] - (subhalo_hmr[index_milkyway_like]*zoom_out_factor), 
                 subhalo_cms[index_milkyway_like][2] + (subhalo_hmr[index_milkyway_like]*zoom_out_factor)]

    
    masses = masses[coordinates[:, 0] < zoom_in_x[1]]
    coordinates = coordinates[coordinates[:, 0] < zoom_in_x[1]]
    
    masses = masses[coordinates[:, 0] > zoom_in_x[0]]
    coordinates = coordinates[coordinates[:, 0] > zoom_in_x[0]]
    
    masses = masses[coordinates[:, 1] < zoom_in_y[1]]
    coordinates = coordinates[coordinates[:, 1] < zoom_in_y[1]]
    
    masses = masses[coordinates[:, 1] > zoom_in_y[0]]
    coordinates = coordinates[coordinates[:, 1] > zoom_in_y[0]]
    
    masses = masses[coordinates[:, 2] < zoom_in_z[1]]
    coordinates = coordinates[coordinates[:, 2] < zoom_in_z[1]]
    
    masses = masses[coordinates[:, 2] > zoom_in_z[0]]
    coordinates = coordinates[coordinates[:, 2] > zoom_in_z[0]]

    ############################################
    
    # calculate the hist
    h, xedges, yedges = np.histogram2d(coordinates[:, 0], coordinates[:, 1], weights=masses, bins=resolution)
    
    if mask_zeros:
        h[h == 0] = mask_zeros_value
        norm = mpl.colors.LogNorm(vmin=h.min(), vmax=h.max())
    else:
        norm = mpl.colors.LogNorm(vmax=h.max())

    # plot
    fig, ax = plt.subplots(dpi=150)
    img = ax.imshow(h.T, origin='lower', cmap='magma',
                    extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
                    norm=norm, interpolation='nearest')

    cbar = plt.colorbar(img, ax=ax)
    
    # make colorbar nice
    ticks = cbar.get_ticks()
    new_labels = [f"{tick/physical_bin_volume:.1e}" for tick in ticks]
    cbar.set_ticks(ticks)
    cbar.set_ticklabels(new_labels)
    cbar.set_label(r"Density [$M_\odot$/$Mpc^3$]")
    
    ax.set_facecolor('black')
    
    ax.set_xlabel('x [ckpc/h]')
    ax.set_ylabel('y [ckpc/h]')
    ax.set_title('2D Projection Histrogramm')
    plt.show()

In [ ]:
path = "/vera/u/jerbo/my_ptmp/L25n128_suite/gridpoint0/"
snapN = 5

subhalo_mass = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloMass") * 1e10 # in M_sun
M_milkyway = 1.2e12

index_milkyway_like = min(range(len(subhalo_mass)), key=lambda i: abs(subhalo_mass[i] - M_milkyway))
M_milkyway_like = subhalo_mass[index_milkyway_like]
print(index_milkyway_like, M_milkyway_like)

subhalo_cms = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloCM")
print(subhalo_cms[index_milkyway_like])

subhalo_hmr = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloHalfmassRad")
print(subhalo_hmr[index_milkyway_like])

box_size = get_data_from_header(path, snapN, 'BoxSize')/1e3 # in Mpc

In [ ]:
coordinates = get_data_from_snap_folder(path, snapN, "PartType4", "Coordinates")
print(coordinates)

In [ ]:
zoom_out_factor = 10
zoom_in_x = [subhalo_cms[index_milkyway_like][0] - (subhalo_hmr[index_milkyway_like]*zoom_out_factor), 
             subhalo_cms[index_milkyway_like][0] + (subhalo_hmr[index_milkyway_like]*zoom_out_factor)]
zoom_in_y = [subhalo_cms[index_milkyway_like][1] - (subhalo_hmr[index_milkyway_like]*zoom_out_factor), 
             subhalo_cms[index_milkyway_like][1] + (subhalo_hmr[index_milkyway_like]*zoom_out_factor)]
zoom_in_z = [subhalo_cms[index_milkyway_like][2] - (subhalo_hmr[index_milkyway_like]*zoom_out_factor), 
             subhalo_cms[index_milkyway_like][2] + (subhalo_hmr[index_milkyway_like]*zoom_out_factor)]

print(box_size*1e3)

In [ ]:
print(coordinates.shape)
print(coordinates)
coordinates = coordinates[coordinates[:, 0] < zoom_in_x[1]]
coordinates = coordinates[coordinates[:, 0] > zoom_in_x[0]]
coordinates = coordinates[coordinates[:, 1] < zoom_in_y[1]]
coordinates = coordinates[coordinates[:, 1] > zoom_in_y[0]]
coordinates = coordinates[coordinates[:, 2] < zoom_in_z[1]]
coordinates = coordinates[coordinates[:, 2] > zoom_in_z[0]]
print(coordinates.shape)
print(coordinates[:10])

In [ ]:
print(get_data_from_header(path, snapN, 'MassTable'))

In [ ]:
path = "/vera/u/jerbo/my_ptmp/L25n256_suite/gridpoint25/"
simple_hist_plot_old(path, 5, particle_type="PartType4", resolution=512, zoom_out_factor=1)

In [ ]:
# print(sorted(subhalo_mass, reverse=True))

In [ ]:
def subhalo_projection(path, snapN, particle_type="PartType4", zomm_in_box_size=500, look_for_centeral_bh_radius=500):
    subhalo_mass = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloMass") * 1e10 # in M_sun

    index_biggeset_halo = max(range(len(subhalo_mass)), key=lambda i: subhalo_mass[i])
    bigges_subhalo = subhalo_mass[index_biggeset_halo]

    subhalo_cms = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloCM")
    biggest_subhalo_cms = subhalo_cms[index_biggeset_halo]
    
    centeral_galaxy_bh = find_centeral_bh(path, snapN, biggest_subhalo_cms, look_for_centeral_bh_radius)

    boxsize = get_data_from_header(path, snapN, 'BoxSize') # in kpc
    
    # transform coordinates to center biggest_halo_cms
    coordinates = get_data_from_snap_folder(path, snapN, particle_type, "Coordinates")
    masses = get_data_from_snap_folder(path, snapN, particle_type, "Masses")*1e10 # in M_sun

    # transform to periodic coordinates centered around centeral_galaxy_bh
    cms_center_coordinates = coordinates - centeral_galaxy_bh
    cms_center_coordinates = (cms_center_coordinates + boxsize / 2) % boxsize - boxsize / 2
    
    masses = masses[cms_center_coordinates[:, 0] < zomm_in_box_size]
    cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 0] < zomm_in_box_size]
    masses = masses[cms_center_coordinates[:, 0] > -zomm_in_box_size]
    cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 0] > -zomm_in_box_size]
    masses = masses[cms_center_coordinates[:, 1] < zomm_in_box_size]
    cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 1] < zomm_in_box_size]
    masses = masses[cms_center_coordinates[:, 1] > -zomm_in_box_size]
    cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 1] > -zomm_in_box_size]
    masses = masses[cms_center_coordinates[:, 2] < zomm_in_box_size]
    cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 2] < zomm_in_box_size]
    masses = masses[cms_center_coordinates[:, 2] > -zomm_in_box_size]
    cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 2] > -zomm_in_box_size]
    
    
    return cms_center_coordinates, masses

In [ ]:
def find_centeral_bh(path, snapN, zoom_in_box_middle, zoom_in_box_size):
    particle_type = "PartType5"
    coordinates = get_data_from_snap_folder(path, snapN, particle_type, "Coordinates")
    boxsize = get_data_from_header(path, snapN, 'BoxSize') # in kpc
    
    # transform into periodic box with center at zoom_in_box_middle
    center_coordinates = (coordinates - zoom_in_box_middle + boxsize / 2) % boxsize - boxsize / 2
    
    # only look at objects within a box centered around 0 with zoom_in_box_size as length, heigt and width 
    center_coordinates = center_coordinates[center_coordinates[:, 0] < zoom_in_box_size]
    center_coordinates = center_coordinates[center_coordinates[:, 0] > -zoom_in_box_size]
    center_coordinates = center_coordinates[center_coordinates[:, 1] < zoom_in_box_size]
    center_coordinates = center_coordinates[center_coordinates[:, 1] > -zoom_in_box_size]
    center_coordinates = center_coordinates[center_coordinates[:, 2] < zoom_in_box_size]
    center_coordinates = center_coordinates[center_coordinates[:, 2] > -zoom_in_box_size]
    
    # center_coordinates is ordered by mass, so center_coordinates[0] is the most massive BH in the box
    # then transform back to original coordinates
    centeral_bh_coords = (center_coordinates[0] + zoom_in_box_middle + boxsize / 2) % boxsize - boxsize / 2
    
    return centeral_bh_coords

In [ ]:
path = "/vera/u/jerbo/my_ptmp/L25n256_suite/gridpoint0/"
snapN = 5
particle_type = "PartType4"
resolution = 256

subhalo_mass = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloMass") * 1e10 # in M_sun

index_biggeset_halo = max(range(len(subhalo_mass)), key=lambda i: subhalo_mass[i])
bigges_subhalo = subhalo_mass[index_milkyway_like]
print(index_biggeset_halo, bigges_subhalo)

subhalo_cms = get_data_from_fof_folder(path, snapN, "Subhalo", "SubhaloCM")
biggest_subhalo_cms = subhalo_cms[index_biggeset_halo]
print(biggest_subhalo_cms)

centeral_galaxy_bh = find_centeral_bh(path, snapN, biggest_subhalo_cms, 500)

boxsize = get_data_from_header(path, snapN, 'BoxSize') # in kpc

In [ ]:
# transform coordinates to center biggest_halo_cms
coordinates = get_data_from_snap_folder(path, snapN, particle_type, "Coordinates")
masses = get_data_from_snap_folder(path, snapN, particle_type, "Masses")*1e10 # in M_sun

box_size = get_data_from_header(path, snapN, 'BoxSize')/1e3 # in Mpc
physical_bin_volume = (box_size/resolution)**2 * box_size  # this is a projection, so one axis has the full length of the box

############# Zoom in Filter ###############
print(coordinates)
cms_center_coordinates = coordinates - centeral_galaxy_bh
print(cms_center_coordinates)
cms_center_coordinates = (cms_center_coordinates + boxsize / 2) % boxsize - boxsize / 2
cms_center_point = np.array([0, 0, 0])
print(cms_center_coordinates)

In [ ]:
zomm_in_box_size = 500

masses = masses[cms_center_coordinates[:, 0] < zomm_in_box_size]
cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 0] < zomm_in_box_size]
masses = masses[cms_center_coordinates[:, 0] > -zomm_in_box_size]
cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 0] > -zomm_in_box_size]
masses = masses[cms_center_coordinates[:, 1] < zomm_in_box_size]
cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 1] < zomm_in_box_size]
masses = masses[cms_center_coordinates[:, 1] > -zomm_in_box_size]
cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 1] > -zomm_in_box_size]
masses = masses[cms_center_coordinates[:, 2] < zomm_in_box_size]
cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 2] < zomm_in_box_size]
masses = masses[cms_center_coordinates[:, 2] > -zomm_in_box_size]
cms_center_coordinates = cms_center_coordinates[cms_center_coordinates[:, 2] > -zomm_in_box_size]

In [ ]:
cms_stars = np.sum(cms_center_coordinates*masses[:, np.newaxis], axis=0)/np.sum(masses)
print(cms_stars)

In [ ]:
# calculate the hist
h, xedges, yedges = np.histogram2d(cms_center_coordinates[:, 0], cms_center_coordinates[:, 1], weights=masses, bins=resolution)

norm = mpl.colors.LogNorm(vmax=h.max())

# plot
fig, ax = plt.subplots(dpi=150)
img = ax.imshow(h.T, origin='lower', cmap='magma',
                extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
                norm=norm, interpolation='nearest')

ax.scatter(cms_center_point[0], cms_center_point[1], c="green", marker="x")
ax.scatter(cms_stars[0], cms_stars[1], c="yellow", marker="x")
# ax.scatter(centeral_galaxy_bh[0], centeral_galaxy_bh[1], c="blue", marker="x")

cbar = plt.colorbar(img, ax=ax)

# make colorbar nice
ticks = cbar.get_ticks()
new_labels = [f"{tick/physical_bin_volume:.1e}" for tick in ticks]
cbar.set_ticks(ticks)
cbar.set_ticklabels(new_labels)
cbar.set_label(r"Density [$M_\odot$/$Mpc^3$]")

ax.set_facecolor('black')

ax.set_xlabel('x [ckpc/h]')
ax.set_ylabel('y [ckpc/h]')
ax.set_title('2D Projection Histrogramm')
plt.show()

In [ ]:
path = "/vera/u/jerbo/my_ptmp/L25n256_suite/gridpoint7/"
snapN = 5
gas_resolution=64
stars_resolution = 128
zoom_box_size = 1000
ref_line_length = 200


zoom_box_half = zoom_box_size/2

coordinates_stars, masses_stars = subhalo_projection(path, snapN, particle_type="PartType4", zomm_in_box_size=zoom_box_half,
                                         look_for_centeral_bh_radius=300)

coordinates_gas, masses_gas = subhalo_projection(path, snapN, particle_type="PartType0", zomm_in_box_size=zoom_box_half,
                                         look_for_centeral_bh_radius=300)

# calculate the hist
h_gas, xedges_gas, yedges_gas = np.histogram2d(coordinates_gas[:, 0], coordinates_gas[:, 1], weights=masses_gas, bins=gas_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
h_stars, xedges_stars, yedges_stars = np.histogram2d(coordinates_stars[:, 0], coordinates_stars[:, 1], weights=masses_stars, bins=stars_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])



# plot
fig, ax = plt.subplots(nrows=2, ncols=1, dpi=500)

# gas
norm_gas = mpl.colors.LogNorm(vmax=h_gas.max())
img_gas = ax[0].imshow(h_gas.T, origin='lower', cmap='magma_r',
                extent=[xedges_gas[0], xedges_gas[-1], yedges_gas[0], yedges_gas[-1]],
                    norm=norm_gas, interpolation='nearest')

ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
ax[0].plot(ref_line[0], ref_line[1], c="white")
ax[0].text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

# Colorbar
cbar_gas = plt.colorbar(img_gas, ax=ax[0])
ticks_gas = cbar_gas.get_ticks()
physical_bin_volume_gas = (zoom_box_size/gas_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
new_labels_gas = [f"{ticks_gas/physical_bin_volume_gas:.1e}" for ticks_gas in ticks_gas]
cbar_gas.set_ticks(ticks_gas)
cbar_gas.set_ticklabels(new_labels_gas)
cbar_gas.set_label(r"Density [$M_\odot$/$Mpc^3$]")

ax[0].set_facecolor('black')

ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)

# stars
norm_stars = mpl.colors.LogNorm(vmax=h_stars.max())
img_stars = ax[1].imshow(h_stars.T, origin='lower', cmap='magma',
                extent=[xedges_stars[0], xedges_stars[-1], yedges_stars[0], yedges_stars[-1]],
                      norm=norm_stars, interpolation='nearest')

ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
ax[1].plot(ref_line[0], ref_line[1], c="white")
ax[1].text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

# Colorbar
cbar_stars = plt.colorbar(img_stars, ax=ax[1])
ticks_stars = cbar_stars.get_ticks()
physical_bin_volume_stars = (zoom_box_size/stars_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
new_labels_stars = [f"{ticks_stars/physical_bin_volume_stars:.1e}" for ticks_stars in ticks_stars]
cbar_stars.set_ticks(ticks_stars)
cbar_stars.set_ticklabels(new_labels_stars)
cbar_stars.set_label(r"Density [$M_\odot$/$Mpc^3$]")

ax[1].set_facecolor('black')

ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)

plt.savefig("plots/test_galaxy_show.pdf", format="PDF")
plt.show()

In [ ]:
print(xedges_stars[0], xedges_stars[-1], yedges_stars[0], yedges_stars[-1])

In [ ]:
def get_cosmo_parameters(basepath):
    path = basepath+"output/txt-files/parameters-usedvalues"
    Omega0 = None
    OmegaLambda = None
    HubbleParam = None
    OmegaBaryon = None
    
    with open(path, "r") as f:
        for line in f:
            if "Omega0" in line:
                Omega0 = float(line.split()[-1])
            if "OmegaBaryon" in line:
                OmegaBaryon = float(line.split()[-1])
            if "OmegaLambda" in line:
                OmegaLambda = float(line.split()[-1])
            if "HubbleParam" in line:
                HubbleParam = float(line.split()[-1])
    
    return Omega0, OmegaBaryon, OmegaLambda, HubbleParam

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/"
snapN = 3
gas_resolution=64
stars_resolution = 128
zoom_box_size = 1000
ref_line_length = 200

gps_to_use = [i for i in range(1, 15)]
n_cols = 7
n_rows = 2

zoom_box_half = zoom_box_size/2

# get coords and masses for all boxes
star_hists = []
gas_hists = []
for gp_counter in range(n_cols*n_rows):
    print(f"gp {gp_counter} calcs...")
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    
    cs_temp, ms_temp = subhalo_projection(path_gp, snapN, particle_type="PartType4", zomm_in_box_size=zoom_box_half,
                                         look_for_centeral_bh_radius=300)
    cg_temp, mg_temp = subhalo_projection(path_gp, snapN, particle_type="PartType0", zomm_in_box_size=zoom_box_half,
                                             look_for_centeral_bh_radius=300)
    
    h_stars, xedges_stars, yedges_stars = np.histogram2d(cs_temp[:, 0], cs_temp[:, 1], weights=ms_temp, bins=stars_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
    
    h_gas, xedges_gas, yedges_gas = np.histogram2d(cg_temp[:, 0], cg_temp[:, 1], weights=mg_temp, bins=gas_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
    
    star_hists.append(h_stars)
    gas_hists.append(h_gas)
    
max_star_norm_1 = max([i.max() for i in star_hists[:n_cols]])
max_gas_norm_1 = max([i.max() for i in gas_hists[:n_cols]])

norm_stars_1 = mpl.colors.LogNorm(vmax=max_star_norm_1)
norm_gas_1 = mpl.colors.LogNorm(vmax=max_gas_norm_1)

max_star_norm_2 = max([i.max() for i in star_hists[n_cols:]])
max_gas_norm_2 = max([i.max() for i in gas_hists[n_cols:]])

norm_stars_2 = mpl.colors.LogNorm(vmax=max_star_norm_2)
norm_gas_2 = mpl.colors.LogNorm(vmax=max_gas_norm_2)


fig = plt.figure(figsize=(14, 8))

# Gesamtes Grid: 2 Blöcke (je 2 Reihen) × 5 Spalten
outer = gridspec.GridSpec(n_rows, 1, hspace=0.15)  # Abstand zwischen Block1 & Block2

# Block 1: Reihe 1 & 2 ohne Abstand
block1 = gridspec.GridSpecFromSubplotSpec(2, n_cols, subplot_spec=outer[0], hspace=0, wspace=0)
gp_counter = 0
for i in range(n_cols):
    ax_gas = fig.add_subplot(block1[0, i])
    ax_stars = fig.add_subplot(block1[1, i])
    
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    print(path_gp)
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path_gp)

    # calculate the hist
    h_gas = gas_hists[gp_counter]
    h_stars = star_hists[gp_counter]
    
    # gas
    
    img_gas = ax_gas.imshow(h_gas.T, origin='lower', cmap='magma_r',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                        norm=norm_gas_1, interpolation='nearest')

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_gas.plot(ref_line[0], ref_line[1], c="white")
    ax_gas.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')
    
    if i == n_cols-1:
        # Colorbar
        cbar_gas = fig.colorbar(img_gas, ax=ax_gas, fraction=0.046, pad=0)
        ticks_gas = cbar_gas.get_ticks()
        physical_bin_volume_gas = (zoom_box_size/gas_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_gas = [f"{ticks_gas/physical_bin_volume_gas:.1e}" for ticks_gas in ticks_gas]
        cbar_gas.set_ticks(ticks_gas)
        cbar_gas.set_ticklabels(new_labels_gas)
        cbar_gas.minorticks_on()
        cbar_gas.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_gas.set_facecolor('black')

    ax_gas.get_xaxis().set_visible(False)
    ax_gas.get_yaxis().set_visible(False)
    
    ax_gas.set_title(fr"$\Omega_m$={Omega0:.2f}, $\Omega_b$={OmegaBaryon:.2f}"+ "\n"+ fr"$\Omega_\Lambda$={OmegaLambda:.2f}, h={HubbleParam:.2f}", fontsize=8)

    # stars
    img_stars = ax_stars.imshow(h_stars.T, origin='lower', cmap='magma',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                          norm=norm_stars_1, interpolation='nearest')

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_stars.plot(ref_line[0], ref_line[1], c="white")
    ax_stars.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

    if i == n_cols-1:
        # Colorbar
        cbar_stars = plt.colorbar(img_stars, ax=ax_stars, fraction=0.046, pad=0)
        ticks_stars = cbar_stars.get_ticks()
        physical_bin_volume_stars = (zoom_box_size/stars_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_stars = [f"{ticks_stars/physical_bin_volume_stars:.1e}" for ticks_stars in ticks_stars]
        cbar_stars.set_ticks(ticks_stars)
        cbar_stars.set_ticklabels(new_labels_stars)
        cbar_stars.minorticks_on()
        cbar_stars.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_stars.set_facecolor('black')

    ax_stars.get_xaxis().set_visible(False)
    ax_stars.get_yaxis().set_visible(False)
    
    gp_counter += 1


    

# Block 2: Reihe 3 & 4 ohne Abstand
block2 = gridspec.GridSpecFromSubplotSpec(2, n_cols, subplot_spec=outer[1], hspace=0, wspace=0)
for i in range(n_cols):
    ax_gas = fig.add_subplot(block2[0, i])
    ax_stars = fig.add_subplot(block2[1, i])
    
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    print(path_gp)
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path_gp)
    
    # calculate the hist
    h_gas = gas_hists[gp_counter]
    h_stars = star_hists[gp_counter]
    
    # gas
    
    img_gas = ax_gas.imshow(h_gas.T, origin='lower', cmap='magma_r',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                        norm=norm_gas_2, interpolation='nearest')

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_gas.plot(ref_line[0], ref_line[1], c="white")
    ax_gas.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')
    
    if i == n_cols-1:
        # Colorbar
        cbar_gas = fig.colorbar(img_gas, ax=ax_gas, fraction=0.046, pad=0)
        ticks_gas = cbar_gas.get_ticks()
        physical_bin_volume_gas = (zoom_box_size/gas_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_gas = [f"{ticks_gas/physical_bin_volume_gas:.1e}" for ticks_gas in ticks_gas]
        cbar_gas.set_ticks(ticks_gas)
        cbar_gas.set_ticklabels(new_labels_gas)
        cbar_gas.minorticks_on()
        cbar_gas.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_gas.set_facecolor('black')

    ax_gas.get_xaxis().set_visible(False)
    ax_gas.get_yaxis().set_visible(False)
    
    ax_gas.set_title(fr"$\Omega_m$={Omega0:.2f}, $\Omega_b$={OmegaBaryon:.2f}"+ "\n"+ fr"$\Omega_\Lambda$={OmegaLambda:.2f}, h={HubbleParam:.2f}", fontsize=8)

    # stars
    img_stars = ax_stars.imshow(h_stars.T, origin='lower', cmap='magma',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                          norm=norm_stars_2, interpolation='nearest')

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_stars.plot(ref_line[0], ref_line[1], c="white")
    ax_stars.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

    if i == n_cols-1:
        # Colorbar
        cbar_stars = plt.colorbar(img_stars, ax=ax_stars, fraction=0.046, pad=0)
        ticks_stars = cbar_stars.get_ticks()
        physical_bin_volume_stars = (zoom_box_size/stars_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_stars = [f"{ticks_stars/physical_bin_volume_stars:.1e}" for ticks_stars in ticks_stars]
        cbar_stars.set_ticks(ticks_stars)
        cbar_stars.set_ticklabels(new_labels_stars)
        cbar_stars.minorticks_on()
        cbar_stars.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_stars.set_facecolor('black')

    ax_stars.get_xaxis().set_visible(False)
    ax_stars.get_yaxis().set_visible(False)
    
    gp_counter += 1

    
plt.savefig("plots/galaxy_galery_test.pdf", format="PDF", dpi=500)
plt.show()

In [ ]:
base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/"
snapN = 5
gas_resolution=64
stars_resolution = 128
zoom_box_size = 1000
ref_line_length = 200

gps_to_use = [i for i in range(1, 15)]
n_cols = 7
n_rows = 2

zoom_box_half = zoom_box_size/2

ref_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/gridpoint0/"
subhalo_mass = get_data_from_fof_folder(ref_path, snapN, "Group", "GroupMass")
index_biggest_halo = max(range(len(subhalo_mass)), key=lambda i: subhalo_mass[i])
reference_halo_cms = get_data_from_fof_folder(ref_path, snapN, "Group", "GroupCM")
reference_halo_cms = reference_halo_cms[index_biggest_halo]  # most massive halo in gp 0

# get coords and masses for all boxes
star_hists = []
star_halfmass_radii = []
gas_hists = []
gas_halfmass_radii = []
for gp_counter in range(n_cols*n_rows):
    print(f"gp {gp_counter} calcs...")
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    
    cs_temp, ms_temp, rs_temp = subhalo_projection(path_gp, snapN, particle_type="PartType4", zomm_in_box_size=zoom_box_half,
                                         look_for_centeral_bh_radius=500, reference_subhalo_cms=reference_halo_cms)
    cg_temp, mg_temp, rg_temp = subhalo_projection(path_gp, snapN, particle_type="PartType0", zomm_in_box_size=zoom_box_half,
                                             look_for_centeral_bh_radius=500, reference_subhalo_cms=reference_halo_cms)
    
    h_stars, xedges_stars, yedges_stars = np.histogram2d(cs_temp[:, 0], cs_temp[:, 1], weights=ms_temp, bins=stars_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
    
    h_gas, xedges_gas, yedges_gas = np.histogram2d(cg_temp[:, 0], cg_temp[:, 1], weights=mg_temp, bins=gas_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
    
    star_halfmass_radii.append(rs_temp)
    star_hists.append(h_stars)
    gas_halfmass_radii.append(rg_temp)
    gas_hists.append(h_gas)
    
max_star_norm_1 = max([i.max() for i in star_hists[:n_cols]])
max_gas_norm_1 = max([i.max() for i in gas_hists[:n_cols]])

norm_stars_1 = mpl.colors.LogNorm(vmax=max_star_norm_1)
norm_gas_1 = mpl.colors.LogNorm(vmax=max_gas_norm_1)

max_star_norm_2 = max([i.max() for i in star_hists[n_cols:]])
max_gas_norm_2 = max([i.max() for i in gas_hists[n_cols:]])

norm_stars_2 = mpl.colors.LogNorm(vmax=max_star_norm_2)
norm_gas_2 = mpl.colors.LogNorm(vmax=max_gas_norm_2)


fig = plt.figure(figsize=(14, 8))

# Gesamtes Grid: 2 Blöcke (je 2 Reihen) × 5 Spalten
outer = gridspec.GridSpec(n_rows, 1, hspace=0.15)  # Abstand zwischen Block1 & Block2

# Block 1: Reihe 1 & 2 ohne Abstand
block1 = gridspec.GridSpecFromSubplotSpec(2, n_cols, subplot_spec=outer[0], hspace=0, wspace=0)
gp_counter = 0
for i in range(n_cols):
    ax_gas = fig.add_subplot(block1[0, i])
    ax_stars = fig.add_subplot(block1[1, i])
    
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    print(path_gp)
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path_gp)

    # calculate the hist
    h_gas = gas_hists[gp_counter]
    h_stars = star_hists[gp_counter]

    halfmass_rad_gas = gas_halfmass_radii[gp_counter]
    halfmass_rad_stars = star_halfmass_radii[gp_counter]
    
    # gas
    
    img_gas = ax_gas.imshow(h_gas.T, origin='lower', cmap='magma_r',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                        norm=norm_gas_1, interpolation='nearest')
    
    circle_x, circle_y = create_circle(halfmass_rad_gas)
    ax_gas.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_gas.plot(ref_line[0], ref_line[1], c="white")
    ax_gas.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

    if i == n_cols-1:
        # Colorbar
        cbar_gas = fig.colorbar(img_gas, ax=ax_gas, fraction=0.046, pad=0)
        ticks_gas = cbar_gas.get_ticks()
        physical_bin_volume_gas = (zoom_box_size/gas_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_gas = [f"{ticks_gas/physical_bin_volume_gas:.1e}" for ticks_gas in ticks_gas]
        cbar_gas.set_ticks(ticks_gas)
        cbar_gas.set_ticklabels(new_labels_gas)
        cbar_gas.minorticks_on()
        cbar_gas.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_gas.set_facecolor('black')

    ax_gas.get_xaxis().set_visible(False)
    ax_gas.get_yaxis().set_visible(False)

    ax_gas.set_xlim([-zoom_box_half, zoom_box_half])
    ax_gas.set_ylim([-zoom_box_half, zoom_box_half])
    
    ax_gas.set_title(fr"$\Omega_m$={Omega0:.2f}, $\Omega_b$={OmegaBaryon:.2f}"+ "\n"+ fr"$\Omega_\Lambda$={OmegaLambda:.2f}, h={HubbleParam:.2f}", fontsize=8)

    # stars
    img_stars = ax_stars.imshow(h_stars.T, origin='lower', cmap='magma',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                          norm=norm_stars_1, interpolation='nearest')
    
    circle_x, circle_y = create_circle(halfmass_rad_stars)
    ax_stars.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_stars.plot(ref_line[0], ref_line[1], c="white")
    ax_stars.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

    if i == n_cols-1:
        # Colorbar
        cbar_stars = plt.colorbar(img_stars, ax=ax_stars, fraction=0.046, pad=0)
        ticks_stars = cbar_stars.get_ticks()
        physical_bin_volume_stars = (zoom_box_size/stars_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_stars = [f"{ticks_stars/physical_bin_volume_stars:.1e}" for ticks_stars in ticks_stars]
        cbar_stars.set_ticks(ticks_stars)
        cbar_stars.set_ticklabels(new_labels_stars)
        cbar_stars.minorticks_on()
        cbar_stars.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_stars.set_facecolor('black')

    ax_stars.get_xaxis().set_visible(False)
    ax_stars.get_yaxis().set_visible(False)

    ax_stars.set_xlim([-zoom_box_half, zoom_box_half])
    ax_stars.set_ylim([-zoom_box_half, zoom_box_half])
    
    gp_counter += 1


    

# Block 2: Reihe 3 & 4 ohne Abstand
block2 = gridspec.GridSpecFromSubplotSpec(2, n_cols, subplot_spec=outer[1], hspace=0, wspace=0)
for i in range(n_cols):
    ax_gas = fig.add_subplot(block2[0, i])
    ax_stars = fig.add_subplot(block2[1, i])
    
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    print(path_gp)
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path_gp)
    
    # calculate the hist
    h_gas = gas_hists[gp_counter]
    h_stars = star_hists[gp_counter]

    halfmass_rad_gas = gas_halfmass_radii[gp_counter]
    halfmass_rad_stars = star_halfmass_radii[gp_counter]
    
    # gas
    
    img_gas = ax_gas.imshow(h_gas.T, origin='lower', cmap='magma_r',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                        norm=norm_gas_2, interpolation='nearest')
    
    circle_x, circle_y = create_circle(halfmass_rad_gas)
    ax_gas.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_gas.plot(ref_line[0], ref_line[1], c="white")
    ax_gas.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

    if i == n_cols-1:
        # Colorbar
        cbar_gas = fig.colorbar(img_gas, ax=ax_gas, fraction=0.046, pad=0)
        ticks_gas = cbar_gas.get_ticks()
        physical_bin_volume_gas = (zoom_box_size/gas_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_gas = [f"{ticks_gas/physical_bin_volume_gas:.1e}" for ticks_gas in ticks_gas]
        cbar_gas.set_ticks(ticks_gas)
        cbar_gas.set_ticklabels(new_labels_gas)
        cbar_gas.minorticks_on()
        cbar_gas.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_gas.set_facecolor('black')

    ax_gas.get_xaxis().set_visible(False)
    ax_gas.get_yaxis().set_visible(False)

    ax_gas.set_xlim([-zoom_box_half, zoom_box_half])
    ax_gas.set_ylim([-zoom_box_half, zoom_box_half])
    
    ax_gas.set_title(fr"$\Omega_m$={Omega0:.2f}, $\Omega_b$={OmegaBaryon:.2f}"+ "\n"+ fr"$\Omega_\Lambda$={OmegaLambda:.2f}, h={HubbleParam:.2f}", fontsize=8)

    # stars
    img_stars = ax_stars.imshow(h_stars.T, origin='lower', cmap='magma',
                    extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                          norm=norm_stars_2, interpolation='nearest')
    
    circle_x, circle_y = create_circle(halfmass_rad_stars)
    ax_stars.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

    ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
    ax_stars.plot(ref_line[0], ref_line[1], c="white")
    ax_stars.text(ref_line[0][0], ref_line[1][0]*0.95, fr"{ref_line_length} kpc", va='bottom', fontsize=5, color="white", weight='bold')

    if i == n_cols-1:
        # Colorbar
        cbar_stars = plt.colorbar(img_stars, ax=ax_stars, fraction=0.046, pad=0)
        ticks_stars = cbar_stars.get_ticks()
        physical_bin_volume_stars = (zoom_box_size/stars_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
        new_labels_stars = [f"{ticks_stars/physical_bin_volume_stars:.1e}" for ticks_stars in ticks_stars]
        cbar_stars.set_ticks(ticks_stars)
        cbar_stars.set_ticklabels(new_labels_stars)
        cbar_stars.minorticks_on()
        cbar_stars.set_label(r"Density [$M_\odot$/$Mpc^3$]")

    ax_stars.set_facecolor('black')

    ax_stars.get_xaxis().set_visible(False)
    ax_stars.get_yaxis().set_visible(False)

    ax_stars.set_xlim([-zoom_box_half, zoom_box_half])
    ax_stars.set_ylim([-zoom_box_half, zoom_box_half])
    
    gp_counter += 1

    
plt.savefig("plots/galaxy_galery_test.pdf", format="PDF", dpi=500)
plt.show()